    ... looking at a classic FACTORY PATTERN, heavily mixed with DELAYED
    EXECUTION (Lazy Initialisation).

    Mapping this to how you would architect things in Kotlin is actually the
    perfect way to understand Isaac Lab's underlying design philosophy.

    Here is exactly how `func` acts as a factory in this architecture:


---
1. THE BLUEPRINT (The Data Class)
    When you write `cgf_cone = sim_utils.ConeCfg(...)`, you are NOT creating a
    3D cone in the world. You are essentially just instantiating a standard 
    Kotlin `data class`.

    It holds all the configuration variables (radius, height, materials) in a
    lightweight object, completely isolated from the heavy C++ physics engine.


2. THE FACTORY METHOD (`.func`)
    In Isaac Lab, every configuration object has a property called `func`. This
    property is literally a pointer to the specific Python function (the factory
    worker) that knows how to parse that exact blueprint and talk to the
    Omniverse engine to build it.

    If you were writing this in Kotlin, it would look very similar to a
    Companion Object factory or a functional interface:

```Kotlin
// The Kotlin equivalent of what Isaac Lab is doing
val coneBlueprint = ConeCfg(radius = 0.15, height = 0.5)

// Calling the factory method to physically build it at a specific path
coneBlueprint.func.invoke("/World/Objects/Cone1", coneBlueprint)
```


---
WHY DO THEY DO IT THIS WAY?
    Isaac Lab forces you to use this factory pattern because of PARALLEL AI
    TRAINING.

    When you train Reinforcement Learning robots, you don't just spawn one 
    environment; you spawn 4096 identical environments simultaneously. By 
    passing the blueprint (`ConeCfg`) from the execution (`.func`), you can
    define your robot once, pass that single lightweight configuration object
    to a massive cloner function, and have the cloner rapidly fire `.func` 4096
    times across your 5090's vRAM.  

---

In [ ]:
o               .        ___---___                    .
       .              .--\        --.     .     .         .
                    ./.;_.\     __/~ \.
                   /;  / `-'  __\    . \
 .        .       / ,--'     / .   .;   \        |
                 | .|       /       __   |      -O-       .
                |__/    __ |  . ;   \ | . |      |
                |      /  \\_    . ;| \___|
   .    o       |      \  .~\\___,--'     |           .
                 |     | . ; ~~~~\_    __|
    |             \    \   .  .  ; \  /_/   .
   -O-        .    \   /         . |  ~/                  .
    |    .          ~\ \   .      /  /~          o
  .                   ~--___ ; ___--~
                 .          ---         .              -JT

    ... we move away from just placing static objects in a scene and start
    dealing with RIGID OBJECT--entities that actually obey the laws of physics,
    have amss, and collide with things.

    In the context of your Lego-assembling SO-101 arm project, this is 
    foundational. A Lego brick dropping onto a table or being pushed by a 
    gripper is a "Rigid Object." If you don't master how the physics engine 
    tracks and updates their states, your robot will just phase through the
    bricks.

    ...


---
1. THE NEW DEPENDENCIES: WHY DO WE NEED THEM?
    To handle physics, we introduce four new imports:
    - `torch`: Used for high-performance tensor computations. Instead of 
      iterating through objects one by one in Python (which is very slow), Isaac
      Sim uses PyTorch to calculate the positions, velocities, and states of
      thousands of objects simultaneously on the GPU.
    - `math_utils`: Provides mathematical utilities specifically for simulation,
      like randomising the spawn positions of objects.
    - `RigidObjectCfg`: The configuration class. This is where you define the
      "blueprint" of the object (its mass, its 3D mesh, its initial spawn state)
    - `RigidObject`: The actual active class that represents the spawned object
      in the simulation, managing its physical properties and state.      

---